In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

DRIVER_PATH = './chromedriver.exe'

In [12]:
def scrape_pmi_data(path, name):
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get(f'https://www.investing.com/economic-calendar/{path}')
    time.sleep(2)

    botton_cookies = driver.find_element(
        By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
    )
    botton_cookies.click()
    
    #time.sleep(30)
    #botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
    #botton_signup.click()
    time.sleep(2)

    if path[-4] == '-':
        id = path[-3:]
    else:
        id = path[-4:]
    botton_cookies = driver.find_element(
        By.XPATH, f'//*[@id="showMoreHistory{id}"]'
    )

    try:
        for _ in range(200):
            botton_cookies.click()
    except:
        print('Done')

    table = driver.find_element(
        By.XPATH, '//*[@id="eventTabDiv_history_0"]'
    )
    dfs = pd.read_html(table.get_attribute('innerHTML'))
    df_raw = dfs[0]

    for i in range(len(df_raw)):
        if df_raw.iloc[i, 0].endswith(')'):
            df_raw.iloc[i, 0] = df_raw.iloc[i, 0][:-5]    
    df_raw.index = pd.to_datetime(df_raw['Release Date'])
    df_raw = df_raw[~df_raw.index.duplicated()]

    driver.close()
    
    df = pd.DataFrame(index=df_raw.index)
    df['Actual'] = df_raw['Actual']
    df['Forecast'] = df_raw['Forecast']
    df.to_csv(f'Data_01-08\{name}_PMI.csv')
    return df

In [13]:
def get_pmi_data(path, country):
    try:
        df = pd.read_csv(f'Data_01-08\{country}_PMI.csv', index_col=0, parse_dates=True)
    except:
        df = scrape_pmi_data(path=path, name=country)
    return df

In [14]:
get_pmi_data(path='manufacturing-pmi-1201', country='Eurozone')

,Actual,Forecast
Release Date,,
2023-08-01,42.7,42.7
2023-07-24,42.7,43.5
2023-07-03,43.4,43.6
2023-06-23,43.6,44.8
2023-06-01,44.8,44.6
...,...,...
2008-04-23,51.8,51.6
2008-04-01,52.0,52.0
2008-03-20,52.0,52.0


In [15]:
country_to_path = dict()
country_to_path.update({'United States': 'ism-manufacturing-pmi-173'}) # From 1999
country_to_path.update({'Japan': 'manufacturing-pmi-202'}) # From 2008
country_to_path.update({'United Kingdom': 'manufacturing-pmi-204'}) # From 2008
country_to_path.update({'Canada': 'manufacturing-pmi-1029'}) # From 2021
country_to_path.update({'France': 'french-manufacturing-pmi-340'}) # From 2008
country_to_path.update({'Switzerland': ''}) 
country_to_path.update({'Germany': 'german-manufacturing-pmi-136'}) # From 2008
country_to_path.update({'Australia': 'manufacturing-pmi-1838'}) # From 2018
country_to_path.update({'Netherlands': ''}) 
country_to_path.update({'Denmark': ''}) 
country_to_path.update({'Sweden': 'swedish-manufacturing-pmi-798'}) # From 1999
country_to_path.update({'Spain': 'spanish-manufacturing-pmi-667'}) # From 2012
country_to_path.update({'Hong Kong': 'hong-kong-manufacturing-pmi-1088'}) # From 2012
country_to_path.update({'Italy': 'italian-manufacturing-pmi-832'}) # From 2012
country_to_path.update({'Singapore': 'singaporean-pmi-792'}) # From 1999
country_to_path.update({'Finland': ''}) 
country_to_path.update({'Belgium': ''}) 
country_to_path.update({'Norway': 'norwegian-pmi-manufacturing-642'}) # From 2004
country_to_path.update({'Israel': ''}) 
country_to_path.update({'Ireland': 'irish-manufacturing-pmi-1481'}) # From 2012
country_to_path.update({'New Zealand': ''}) 
country_to_path.update({'Austria': 'austrian-manufacturing-pmi-1544'}) # From 2012
country_to_path.update({'Portugal': ''})

In [16]:
df_pmi = pd.DataFrame(index=pd.date_range('1999-01-01', '2023-08-02'))

for country in country_to_path:
    path = country_to_path[country]
    if path != '':
        df_pmi[country] = get_pmi_data(path, country)['Actual']

df_pmi

,United States,Japan,United Kingdom,Canada,France,Germany,Australia,Sweden,Spain,Hong Kong,Italy,Singapore,Norway,Ireland,Austria
1999-01-01,46.8,NaN,NaN,NaN,NaN,NaN,NaN,44.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-31,NaN,49.6,NaN,NaN,NaN,NaN,49.6,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN
2023-08-01,46.4,NaN,45.3,49.6,45.1,38.8,NaN,47.6,47.8,NaN,44.5,NaN,56.7,NaN,NaN


In [17]:
for country in df_pmi.columns:
    first_valid = df_pmi[country].first_valid_index()
    print(country, first_valid)

United States 1999-01-01 00:00:00
Japan 2008-02-28 00:00:00
United Kingdom 2008-03-03 00:00:00
Canada 2021-01-04 00:00:00
France 2008-08-21 00:00:00
Germany 2008-03-03 00:00:00
Australia 2018-12-13 00:00:00
Sweden 1999-01-01 00:00:00
Spain 2012-03-01 00:00:00
Hong Kong 2012-07-01 00:00:00
Italy 2012-07-01 00:00:00
Singapore 1999-02-01 00:00:00
Norway 2004-03-01 00:00:00
Ireland 2012-07-01 00:00:00
Austria 2012-07-01 00:00:00


In [18]:
emerging_country_to_path = dict()
emerging_country_to_path.update({'China': 'chinese-manufacturing-pmi-594'}) # From 2005
emerging_country_to_path.update({'Taiwan': 'taiwanese-manufacturing-pmi-1121'}) # From 2012
emerging_country_to_path.update({'India': 'indian-nikkei-s-p-global-manufacturing-pmi-754'}) # From 2012
emerging_country_to_path.update({'Korea': 'south-korean-nikkei-manufacturing-pmi-785'}) # From 2012
emerging_country_to_path.update({'Brazil': 'brazilian-s-p-global-manufacturing-pmi-1476'}) # From 2012
emerging_country_to_path.update({'Saudi Arabia': ''})
emerging_country_to_path.update({'South Africa': 'south-african-manufacturing-pmi-605'}) # From 2012
emerging_country_to_path.update({'Mexico': 'mexican-manufacturing-pmi-1303'}) # From 2012
emerging_country_to_path.update({'Thailand': ''})
emerging_country_to_path.update({'Indonesia': 'indonesia-nikkei-pmi-1096'}) # From 2012
emerging_country_to_path.update({'Malaysia': ''})
emerging_country_to_path.update({'United Arab Emirates': ''})
emerging_country_to_path.update({'Qatar': ''})
emerging_country_to_path.update({'Kuwait': ''})
emerging_country_to_path.update({'Turkiye': 'turkish-manufacturing-pmi-1305'}) # From 2013
emerging_country_to_path.update({'Philippines': ''})
emerging_country_to_path.update({'Poland': 'polish-manufacturing-pmi-1304'}) # From 2012
emerging_country_to_path.update({'Chile': ''})
emerging_country_to_path.update({'Greece': 'greek-manufacturing-pmi-1472'}) # From 2012
emerging_country_to_path.update({'Peru': ''})
emerging_country_to_path.update({'Hungary': 'hungarian-pmi-674'}) # From 1999
emerging_country_to_path.update({'Czechia': ''})
emerging_country_to_path.update({'Egypt': ''})
emerging_country_to_path.update({'Colombia': ''})
emerging_country_to_path.update({'Argentina': ''})
emerging_country_to_path.update({'Russia': 'russian-s-p-global-manufacturing-pmi-1630'}) # From 2012

In [19]:
df_pmi_em = pd.DataFrame(index=pd.date_range('1999-01-01', '2023-08-02'))

for country in emerging_country_to_path:
    path = emerging_country_to_path[country]
    if path != '':
        df_pmi_em[country] = get_pmi_data(path, country)['Actual']

df_pmi_em

,China,Taiwan,India,Korea,Brazil,South Africa,Mexico,Indonesia,Turkiye,Poland,Greece,Hungary,Russia
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.7,NaN
1999-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-30,49.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-31,NaN,44.1,NaN,49.4,NaN,NaN,NaN,53.3,NaN,NaN,NaN,NaN,NaN
2023-08-01,NaN,NaN,57.7,NaN,47.8,47.3,50.9,NaN,49.9,43.5,53.5,45.7,52.1


In [20]:
for country in df_pmi_em.columns:
    first_valid = df_pmi_em[country].first_valid_index()
    print(country, first_valid)

China 2005-02-01 00:00:00
Taiwan 2012-07-01 00:00:00
India 2012-05-02 00:00:00
Korea 2012-05-01 00:00:00
Brazil 2012-07-01 00:00:00
South Africa 2012-03-01 00:00:00
Mexico 2012-07-01 00:00:00
Indonesia 2012-07-01 00:00:00
Turkiye 2013-02-01 00:00:00
Poland 2012-07-01 00:00:00
Greece 2012-07-01 00:00:00
Hungary 1999-01-01 00:00:00
Russia 2012-07-01 00:00:00


In [21]:
month_start = pd.date_range('2012-07-01', '2023-08-04', freq='MS')
df_pmi_monthly = pd.DataFrame(index=month_start)

df_filled = df_pmi.bfill(limit=10).ffill(limit=15)
for country in df_filled.columns:
    df_pmi_monthly[country] = df_filled[country]

df_filled_em = df_pmi_em.bfill(limit=10).ffill(limit=15)
for country in df_filled_em.columns:
    df_pmi_monthly[country] = df_filled_em[country]

df_pmi_monthly.to_csv('PMI_2012-08-2022.csv')
df_pmi_monthly

,United States,Japan,United Kingdom,Canada,France,Germany,Australia,Sweden,Spain,Hong Kong,...,Korea,Brazil,South Africa,Mexico,Indonesia,Turkiye,Poland,Greece,Hungary,Russia
2012-07-01,49.7,49.9,48.6,NaN,45.2,45.0,NaN,48.4,41.1,49.8,...,49.38,48.5,48.2,55.9,50.2,NaN,48.0,40.1,52.8,51.0
2012-08-01,49.8,47.9,45.4,NaN,43.4,43.0,NaN,50.6,42.3,50.3,...,47.20,48.7,51.0,55.2,51.4,NaN,49.7,41.9,51.9,52.0
2012-09-01,49.6,47.7,49.5,NaN,46.0,44.7,NaN,45.1,44.0,50.5,...,47.50,49.3,50.2,55.1,51.6,NaN,48.3,42.1,49.5,51.0
2012-10-01,51.5,48.0,48.4,NaN,42.7,47.4,NaN,44.7,44.6,49.6,...,45.71,49.8,46.2,54.4,50.5,NaN,47.0,42.2,52.5,52.4
2012-11-01,51.7,46.9,47.5,NaN,43.7,46.0,NaN,43.1,43.5,50.5,...,47.37,50.2,47.1,55.5,51.9,NaN,47.3,41.0,49.9,52.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-01,46.3,49.2,47.9,48.6,47.3,44.7,49.1,45.7,51.3,53.5,...,47.60,47.0,48.1,51.0,51.9,50.9,48.3,52.8,55.3,53.2
2023-05-01,47.1,49.5,47.8,50.2,45.6,44.5,48.0,45.5,49.0,52.4,...,48.10,44.3,49.8,51.1,52.7,51.5,46.6,52.4,61.9,52.6
2023-06-01,46.9,50.6,47.1,49.0,45.7,43.2,48.4,40.6,48.4,50.6,...,48.40,47.1,49.2,50.5,50.3,51.5,47.0,51.5,57.1,53.5
2023-07-01,46.0,49.8,46.5,48.8,46.0,40.6,48.2,44.8,48.0,50.3,...,47.80,46.6,47.6,50.9,52.5,51.5,45.1,51.8,44.2,52.6


In [22]:
for country in df_pmi_monthly.columns:
    first_valid = df_pmi_monthly[country].first_valid_index()
    if str(first_valid) != '2012-07-01 00:00:00':
        print(country, first_valid)
    
    last_valid = df_pmi_monthly[country].last_valid_index()
    if str(last_valid) != '2023-08-01 00:00:00':
        print(country, last_valid)

Canada 2021-01-01 00:00:00
Australia 2019-01-01 00:00:00
Turkiye 2013-02-01 00:00:00
